In [ ]:
# pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [12]:
import time

print(f'today is {time.strftime("%Y-%m-%d")} and the time is {time.strftime("%H:%M:%S")}')

today is 2024-09-29 and the time is 20:30:31


In [15]:
import os
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from datetime import datetime

SCOPES = ['https://www.googleapis.com/auth/calendar']

def get_calendar_service():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return build('calendar', 'v3', credentials=creds)

def google_calendar_crud(action, event_details=None, event_id=None):
    service = get_calendar_service()
    if action == 'create':
        event = service.events().insert(calendarId='primary', body=event_details).execute()
        return f"Event created: {event.get('htmlLink')}"

    elif action == 'update':
        updated_event = service.events().update(calendarId='primary', eventId=event_id, body=event_details).execute()
        return f"Event updated: {updated_event['updated']}"

    elif action == 'delete':
        service.events().delete(calendarId='primary', eventId=event_id).execute()
        return "Event deleted"

    else:
        return "Invalid action. Use 'create', 'update', or 'delete'."


def get_events_for_date(year, month, day):
    service = get_calendar_service()
    date = datetime(year, month, day)
    start = date.replace(hour=0, minute=0, second=0)
    end = date.replace(hour=23, minute=59, second=59)
    events_result = service.events().list(
        calendarId='primary',
        timeMin=start.isoformat() + 'Z',
        timeMax=end.isoformat() + 'Z',
        singleEvents=True,
        orderBy='startTime'
    ).execute()
    return events_result.get('items', [])

def find_event_in_range(event_name, start_date, end_date):
    service = get_calendar_service()
    events_result = service.events().list(
        calendarId='primary',
        timeMin=start_date.isoformat() + 'Z',
        timeMax=end_date.isoformat() + 'Z',
        q=event_name,
        singleEvents=True,
        orderBy='startTime'
    ).execute()
    return events_result.get('items', [])


In [10]:
# new event example
event_details = {
    'summary': 'Test Event',
    'description': 'This is a test event',
    'start': {'dateTime': '2024-05-01T10:00:00', 'timeZone': 'America/New_York'},
    'end': {'dateTime': '2024-05-01T11:00:00', 'timeZone': 'America/New_York'}
}
google_calendar_crud('create', event_details)

# update event example
event_details = {
    'summary': 'Updated Event',
    'description': 'This is an updated event',
    'start': {'dateTime': '2024-05-01T12:00:00', 'timeZone': 'America/New_York'},
    'end': {'dateTime': '2024-05-01T13:00:00', 'timeZone': 'America/New_York'}
}
google_calendar_crud('update', event_details, event_id='1234567890')

# delete event example
google_calendar_crud('delete', event_id='1234567890')

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1017881126609-q28fvovohc5qq73ud4j6vmavckij3e8f.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A42401%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=94slnduZYU9ktb9vdXBJdDiDA5k0Jh&access_type=offline


'Event created: https://www.google.com/calendar/event?eid=dTV0djdnaTV2YTVqY3JlMzg0bHVoZnM5aWcgZmVybmFuZGV6cmFkb3ZpY2hAbQ'

In [13]:
from datetime import timedelta

# Example usage
if __name__ == '__main__':
       # Check for events on a specific date
    events = get_events_for_date(2024, 10, 17)
    print(f"Events on October 17, 2024: {events}")

    # Find an event called what you want in the next X days
    start = datetime.now()
    end = start + timedelta(days=30)
    events = find_event_in_range("", start, end)
    print(len(events))
    print(events[0].keys())


Events on October 17, 2024: [{'kind': 'calendar#event', 'etag': '"3431736202312000"', 'id': 'g8rk4pab21o6o45eec1r038f04', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=ZzhyazRwYWIyMW82bzQ1ZWVjMXIwMzhmMDQgZmVybmFuZGV6cmFkb3ZpY2hAbQ', 'created': '2024-05-16T14:01:41.000Z', 'updated': '2024-05-16T14:01:41.156Z', 'summary': 'Dentista', 'creator': {'email': 'fernandezradovich@gmail.com', 'self': True}, 'organizer': {'email': 'fernandezradovich@gmail.com', 'self': True}, 'start': {'dateTime': '2024-10-17T08:30:00-03:00', 'timeZone': 'America/Argentina/Buenos_Aires'}, 'end': {'dateTime': '2024-10-17T09:30:00-03:00', 'timeZone': 'America/Argentina/Buenos_Aires'}, 'iCalUID': 'g8rk4pab21o6o45eec1r038f04@google.com', 'sequence': 0, 'reminders': {'useDefault': True}, 'eventType': 'default'}]
2
dict_keys(['kind', 'etag', 'id', 'status', 'htmlLink', 'created', 'updated', 'summary', 'creator', 'organizer', 'start', 'end', 'iCalUID', 'sequence', 'reminders', 'eventType'

In [14]:
events

[{'kind': 'calendar#event',
  'etag': '"3455400885910000"',
  'id': '0052vv3ahfmqo0h4cedqdpgrmo',
  'status': 'confirmed',
  'htmlLink': 'https://www.google.com/calendar/event?eid=MDA1MnZ2M2FoZm1xbzBoNGNlZHFkcGdybW8gZmVybmFuZGV6cmFkb3ZpY2hAbQ',
  'created': '2024-09-30T12:47:22.000Z',
  'updated': '2024-09-30T12:47:22.955Z',
  'summary': 'some random enve',
  'creator': {'email': 'fernandezradovich@gmail.com', 'self': True},
  'organizer': {'email': 'fernandezradovich@gmail.com', 'self': True},
  'start': {'dateTime': '2024-10-06T10:00:00-03:00',
   'timeZone': 'America/Argentina/Buenos_Aires'},
  'end': {'dateTime': '2024-10-06T11:00:00-03:00',
   'timeZone': 'America/Argentina/Buenos_Aires'},
  'iCalUID': '0052vv3ahfmqo0h4cedqdpgrmo@google.com',
  'sequence': 0,
  'reminders': {'useDefault': True},
  'eventType': 'default'},
 {'kind': 'calendar#event',
  'etag': '"3431736202312000"',
  'id': 'g8rk4pab21o6o45eec1r038f04',
  'status': 'confirmed',
  'htmlLink': 'https://www.google.com/